<a href="https://colab.research.google.com/github/pnrdgp/SAIL-Bhilai-Steel-Plant-Project/blob/main/fuel_rate_reduction_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Imported necessary libraries
import pandas as pd
import numpy as np
import sys

# Defined the percentile for "Best-in-Class" performance.
BEST_IN_CLASS_PERCENTILE = 0.10

# --- Step 1: Loading and Preparing Data ---

print("--- Loading and Preparing Data ---")
try:
    # Defining the path to the file in the Colab environment
    file_path = 'blastfurnacedata.csv'

    # Read the file into a pandas DataFrame
    raw_df = pd.read_csv(file_path, encoding='utf-8-sig')

    # Rename columns for consistency
    raw_df = raw_df.rename(columns={
        'ï»¿fuel rate': 'fuel_rate', 'fuel rate': 'fuel_rate', 'moisture': 'moisture',
        'co efficiency': 'co_efficiency', 'hot blast temp': 'hot_blast_temp',
        'slag rate': 'slag_rate', 'flame temp': 'flame_temp', 'top pressure': 'top_pressure'
    })

    # Define and clean required columns
    required_cols = ['fuel_rate', 'moisture', 'co_efficiency', 'hot_blast_temp', 'slag_rate', 'flame_temp', 'top_pressure']
    for col in required_cols:
        raw_df[col] = pd.to_numeric(raw_df[col], errors='coerce')
    df = raw_df.dropna(subset=required_cols)

    print("Data loaded and cleaned successfully.\n")

    # --- Step 2: Identify "Best-in-Class" and "Average" Performance ---
    best_fuel_rate_threshold = df['fuel_rate'].quantile(BEST_IN_CLASS_PERCENTILE)
    best_in_class_df = df[df['fuel_rate'] <= best_fuel_rate_threshold]
    average_performance = df.mean()
    best_in_class_performance = best_in_class_df.mean()

    print(f"--- Analysis Setup ---")
    print(f"Total Days in Dataset: {len(df)}")
    print(f"Number of 'Best-in-Class' Days (Top {BEST_IN_CLASS_PERCENTILE*100}%): {len(best_in_class_df)}")
    print(f"Fuel Rate for an Average Day: {average_performance['fuel_rate']:.2f} kg/thm")
    print(f"Average Fuel Rate for a 'Best-in-Class' Day: {best_in_class_performance['fuel_rate']:.2f} kg/thm\n")

    # --- Step 3: Create a Comparison Report ---
    comparison_df = pd.DataFrame({
        'Average Day': average_performance,
        'Best-in-Class Day': best_in_class_performance
    })
    comparison_df['Difference'] = comparison_df['Best-in-Class Day'] - comparison_df['Average Day']
    comparison_df['% Change'] = (comparison_df['Difference'] / comparison_df['Average Day']) * 100

    report_cols = ['fuel_rate', 'hot_blast_temp', 'moisture', 'co_efficiency', 'slag_rate', 'top_pressure', 'flame_temp']
    final_report = comparison_df.loc[report_cols]

    # --- Step 4: Display the Blueprint for Success ---
    avg_fuel_rate = final_report.loc['fuel_rate', 'Average Day']
    best_fuel_rate = final_report.loc['fuel_rate', 'Best-in-Class Day']
    actual_saving = avg_fuel_rate - best_fuel_rate

    print("--- Blueprint for High Efficiency ---")
    print(f"To move from an 'Average Day' to a 'Best-in-Class Day' (saving approx. {actual_saving:.1f} kg/thm of fuel), the furnace was operated with:")
    print("--------------------------------------------------------------------------------")

    for param in report_cols:
        if param == 'fuel_rate': continue

        diff = final_report.loc[param, 'Difference']
        avg_val = final_report.loc[param, 'Average Day']
        best_val = final_report.loc[param, 'Best-in-Class Day']

        action = "HIGHER" if diff > 0 else "LOWER"

        print(f"-> A {abs(diff):.2f} unit {action} {param.replace('_', ' ').title()} (Avg: {avg_val:.2f} vs. Best: {best_val:.2f})")

    print("--------------------------------------------------------------------------------")
    print("\nThis analysis shows the actual, historical parameters that led to the best performance.")

except FileNotFoundError:
    print(f"\nError: Could not find '{file_path}'.")
    print("Please make sure you have uploaded the 'blastfurnacedata.csv' file to your Colab session's main directory.")
except Exception as e:
    print(f"\nAn error occurred during analysis: {e}")


--- Loading and Preparing Data ---
Data loaded and cleaned successfully.

--- Analysis Setup ---
Total Days in Dataset: 234
Number of 'Best-in-Class' Days (Top 10.0%): 25
Fuel Rate for an Average Day: 571.45 kg/thm
Average Fuel Rate for a 'Best-in-Class' Day: 548.06 kg/thm

--- Blueprint for High Efficiency ---
To move from an 'Average Day' to a 'Best-in-Class Day' (saving approx. 23.4 kg/thm of fuel), the furnace was operated with:
--------------------------------------------------------------------------------
-> A 27.19 unit LOWER Hot Blast Temp (Avg: 1149.47 vs. Best: 1122.28)
-> A 1.06 unit HIGHER Moisture (Avg: 7.16 vs. Best: 8.23)
-> A 0.22 unit LOWER Co Efficiency (Avg: 46.04 vs. Best: 45.82)
-> A 18.31 unit LOWER Slag Rate (Avg: 505.67 vs. Best: 487.36)
-> A 0.01 unit HIGHER Top Pressure (Avg: 2.35 vs. Best: 2.36)
-> A 18.24 unit LOWER Flame Temp (Avg: 2181.59 vs. Best: 2163.34)
--------------------------------------------------------------------------------

This analysis sho